# Natural Language Project - Finding the Genre of Movie Plots - Group 11

## Project Setup
If you're running this project in **Google Colab**, make sure to execute the following commands to properly configure the environment.
(These steps are not required if you're running the project locally on your machine.)

In [1]:
!git clone https://github.com/rspecker/NLP.git
%cd NLP

Cloning into 'NLP'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 262 (delta 0), reused 1 (delta 0), pack-reused 258 (from 1)
Receiving objects: 100% (262/262), 16.93 MiB | 18.05 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/NLP


In [2]:
# !pip install -r requirements.txt

In [2]:
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 6.0 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# import project dependencies
import pandas as pd
from modelling.grid import classifiers
from modelling.grid_search import perform_grid_search
from modelling.evaluation import evaluate_model, save_results_to_file, save_confusion_matrix
from preprocessing.embeddings import create_sentence_embeddings
from preprocessing.preproc import create_preprocesssed_dataset
from preprocessing.tfidf import create_train_data_tfidf
from utils import create_train_test_sets
from modelling.information_ret import score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Data Import

In [5]:
# Import data and set column names
df = pd.read_table(
    'train.txt',
    names=['title', 'from', 'genre', 'director', 'plot']
    )

## Data Pre-Processing

In [6]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = create_train_test_sets(
    df, test_size=0.2, random_state=0, y_column='genre'
)

### TF-IDF
TF-IDF converts text into sparse vectors based on word importance across documents, without considering word order or context

In [8]:
# create TF-IDF
vectorizer, x_train_tfidf = create_train_data_tfidf(x_train)

In [ ]:
vectorizer, x_test_tfidf = create_train_data_tfidf(x_test, vectorizer)

### Feature-based Transfer Learning with Sentence Embeddings (Sentence-BERT)
Sentence-BERT generates dense, fixed-length embeddings that capture the semantic meaning of entire sentences, enabling more context-aware comparisons.

In [9]:
# create sentence embeddings for testing data
x_train_sentence_embeddings = create_sentence_embeddings(
    sentences=x_train["plot"].to_list(),
    model="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# create sentence embeddings for testing data
x_test_sentence_embeddings = create_sentence_embeddings(
    sentences=x_test["plot"].to_list(),
    model="all-MiniLM-L6-v2")

## Training

### TF-IDF

In [ ]:
del classifiers["MultinomialNB"]

In [ ]:
for model_name, (model, param_grid) in classifiers.items():
    model_type="word_embeddings"
    # Perform GridSearchCV
    best_model, best_params, best_score = perform_grid_search(
        model, param_grid, model_type,model_name,
        x_train_sentence_embeddings, y_train)

    # Evaluate the model
    test_accuracy, classification_rep, cm = evaluate_model(
        best_model, x_test_sentence_embeddings, y_test)

    # Save the results
    save_results_to_file(
        model_type, model_name, best_params, best_score, test_accuracy, classification_rep)

    # Save confusion matrix as image
    save_confusion_matrix(model_type, model_name, cm, y_test)

### Feature-based Transfer Learning with Sentence Embeddings (Sentence-BERT)

In [ ]:
# del classifiers["MultinomialNB"]

In [ ]:
# for model_name, (model, param_grid) in classifiers.items():
#     model_type="sentence_embeddings"
#     # Perform GridSearchCV
#     best_model, best_params, best_score = perform_grid_search(
#         model, param_grid, model_type,model_name,
#         x_train_sentence_embeddings, y_train)

#     # Evaluate the model
#     test_accuracy, classification_rep, cm = evaluate_model(
#         best_model, x_test_sentence_embeddings, y_test)

#     # Save the results
#     save_results_to_file(
#         model_type, model_name, best_params, best_score, test_accuracy, classification_rep)

#     # Save confusion matrix as image
#     save_confusion_matrix(model_type, model_name, cm, y_test)

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)
x_train_tfidf = vectorizer.fit_transform(x_train["plot"])
x_test_tfidf = vectorizer.transform(x_test["plot"])

model = MultinomialNB()
params, score = tune_hyperparameters(model, mnb_param_grid, x_train_tfidf, y_train)

final_model = MultinomialNB(alpha=params['alpha'],
                            fit_prior=params['fit_prior'])
final_model.fit(x_train_tfidf, y_train)

final_model.score(x_test_tfidf, y_test)
print(f"Test set accuracy: {final_model.score(x_test_tfidf, y_test)}")

model = SVC()
params, score = tune_hyperparameters(model, svc_param_grid, x_train_tfidf, y_train)

final_model = MultinomialNB(C=params['C'],
                            kernel=params['kernel'],
                            # gamma=params['gamma'],
                            # class_weight=params['class_weight']
                            )
final_model.fit(x_train_tfidf, y_train)

final_model.score(x_test_tfidf, y_test)
print(f"Test set accuracy: {final_model.score(x_test_tfidf, y_test)}")